# CT_HeartSubstruct_SMIT Demo Notebook

User supplies input DICOM files, copy one case to `dcmDir` location.

In [ ]:
import os, shutil, subprocess
from glob import glob

In [ ]:
workDir = '/content'
os.chdir(workDir)

In [ ]:
!pip install pyxnat
!pip install "pyCERR[napari] @ git+https://github.com/cerr/pyCERR.git@testing"

In [ ]:
from cerr import plan_container as pc
from cerr.dcm_export import rtstruct_iod

In [ ]:
! git clone https://github.com/cerr/model_installer.git && cd model_installer && ./installer.sh -m 8 -d /content -p C &&

In [ ]:
dcmDir = '/content/session/input_dcm'
niiDir = '/content/session/input_nii'
maskOutDir = '/content/session/output_nii'
dcmOutDir = '/content/session/output_dcm'

In [ ]:
modelDir = '/content/CT_HeartSubstruct_SMIT'
weightsPath = os.path.join(modelDir,'/trained_weights/model_final.pt')

In [ ]:
!mkdir -p {dcmDir} {niiDir} {maskOutDir} {dcmOutDir}

In [ ]:
planC = pc.loadDcmDir(dcmDir = dcmDir)
niiFileName = os.path.join(niiDir,'CT_scan.nii.gz')
scanNum = 0
planC.scan[scanNum].saveNii(niiFileName = niiFileName)

In [ ]:
cmd = 'source ' + os.path.join(modelDir,'conda-pack','bin','activate') + ' && cd ' + modelDir + ' && source bash_run_SMIT_HeartSubstructures.sh ' + niiDir + ' ' + maskOutDir + ' ' +  niiFileName
subprocess.run(cmd, shell=True, executable="/bin/bash")

In [ ]:
labels_dict = {1: 'SMIT_AORTA',2:'SMIT_2',3:'SMIT_3',4:'SMIT_4',5:'SMIT_5',6:'SMIT_6'}
structNumV = range(len(labels_dict))

In [ ]:
outNiiFile = glob(os.path.join(maskOutDir,'*.nii*'))
print(outNiiFile)

In [ ]:
planC = pc.loadNiiStructure(nii_file_name = outNiiFile, assocScanNum = scanNum, planC = planC, labels_dict = labels_dict)

In [ ]:
structName ='CT_HeartSubstruct_SMIT'
rtstructFile = os.path.join(dcmOutDir,'RTSTRUCT.dcm')
seriesDescription = structName
rtstruct_iod.create(structNumV = structNumV, filePath = rtstructFile, planC = planC, seriesOpts = {'SeriesDescription':seriesDescription})